In [1]:
import cobra
from cobra.io import read_sbml_model,load_json_model
from cobra import Model, Reaction, Metabolite
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord

# Extract tRNA from Lactobacillus plantarum WCFS1

In [18]:
# trna_counts = {}
# rna_records = SeqIO.parse( '../LP_HMX2/220668.9.PATRIC.frn', "fasta")
# trna_records=[]
# for r in rna_records:
#     if 'tRNA' in r.description:
#         if '?' in r.description:
#             continue
#         seq = str(r.seq)
#         des = str(r.description).split('   ')[1].strip()
#         if des not in trna_counts:
#             trna_counts[des] = 1
#             name = des + '_0'
#         else:
#             trna_counts[des] += 1
#             name = des + '_' + str( trna_counts[des]-1 )
#         trna_records.append( SeqRecord(Seq(seq), id = "", name="",description=name) )
# SeqIO.write( trna_records, '../LP_HMX2/LP_WCFS1_trna.fa' ,"fasta")        

70

# Extract chaperones

In [17]:
pro_annot = pd.read_csv('../LP_HMX2/BCH.all_annotation.csv')
pro_annot = pro_annot[['Gene_ID','GO_terms','Kegg_Gene_Name','Nr_Description']]
keep_ind=[]
for i in range(len(pro_annot['Gene_ID'])):
    if 'chaper' in list(pro_annot['Nr_Description'])[i]:
        keep_ind.append(i)
    elif list(pro_annot['Kegg_Gene_Name'])[i] in ['tig','groL','groS','dnaK','grpE','dnaJ']:
        keep_ind.append(i)
keep_ind = list(set(keep_ind))
cpr_annot = (pro_annot.iloc[keep_ind]).reset_index().drop(['index'],axis=1)
cpr_annot

,Gene_ID,GO_terms,Kegg_Gene_Name,Nr_Description
0,1_449,cytoplasm,hslO,MULTISPECIES: Hsp33 family molecular chaperone...
1,1_549,ATP binding,"groES, HSPE1",MULTISPECIES: co-chaperone GroES [Lactobacillus]
2,1_550,ATP binding,"groEL, HSPD1",MULTISPECIES: chaperonin GroEL [Lactobacillus]
3,1_1801,protein folding,tig,MULTISPECIES: trigger factor [Lactobacillus]
4,1_2927,metal ion transport,--,MULTISPECIES: copper chaperone [Lactobacillales]
5,1_1716,ATP binding,dnaJ,MULTISPECIES: molecular chaperone DnaJ [Lactob...
6,1_1717,ATP binding,dnaK,MULTISPECIES: molecular chaperone DnaK [Lactob...
7,1_1718,adenyl-nucleotide exchange factor activity,GRPE,co-chaperone GrpE [Lactobacillus plantarum WJL]
8,1_1627,ATP binding,clpB,MULTISPECIES: ATP-dependent chaperone ClpB [La...


In [23]:
chaperones = {'1_449':'hslO','1_549':'groES','1_550':'groEL','1_1801':'tig',
              '1_1716':'dnaJ','1_1717':'dnaK','1_1718':'GRPE','1_1627':'clpB'}
prot_records = SeqIO.parse( '../LP_HMX2/BCH.protein.fa', "fasta")
cpr_records=[]
for r in prot_records:
    if r.id in chaperones.keys():
        ID = '>rba|'+r.id+'|'+chaperones[r.id]
        cpr_records.append( SeqRecord(Seq(str(r.seq)), id = ID, name="",description="") )
SeqIO.write( cpr_records, '../LP_HMX2/chaperones.fasta' ,"fasta")

8

# Extract ribosomal proteins

In [41]:
rb_prots= [ 'rps' + chr(ord('A')+i) for i in range(21) ] + \
       ['sra'] + ['rpl' + chr(ord('A')+i) for i in range(6)] + ['rpl' + chr(ord('I')+i) for i in range(17)] + \
      [ 'rpm' + chr(ord('A')+i) for i in range(10)] + [ 'inf' + chr(ord('A')+i) for i in range(3)] + \
      ['tsf','tuf','fusA','efp','prfA','prfB']

In [61]:
pro_annot = pd.read_csv('../LP_HMX2/BCH.all_annotation.csv')
pro_annot = pro_annot[['Gene_ID','GO_terms','Kegg_Gene_Name','Nr_Description']]
keep_ind=[]
for i in range(len(pro_annot['Gene_ID'])):
    if list(pro_annot['Kegg_Gene_Name'])[i] in ['rimI', 'rimL', 'ysxB']:
        continue 
    if 'ribosomal' in list(pro_annot['Nr_Description'])[i]:
        keep_ind.append(i)
    else:
        for ID in rb_prots:
            if ID in list(pro_annot['Kegg_Gene_Name'])[i]:
                keep_ind.append(i)
keep_ind = list(set(keep_ind) )
rb_annot = (pro_annot.iloc[keep_ind]).reset_index().drop(['index'],axis=1)

In [71]:
names = []
for i in range(len(rb_annot['Kegg_Gene_Name'])):
    temp_name = list( rb_annot['Kegg_Gene_Name'] )[i]
    for rbp in rb_prots:
        if rbp in temp_name:
            temp_name = rbp
    names.append(temp_name)
rb_annot['name'] = names
ID_list = list(rb_annot['Gene_ID'])
rb_annot['Gene_ID'] = [ ID.strip() for ID in ID_list]

In [73]:
prot_records = SeqIO.parse( '../LP_HMX2/BCH.protein.fa', "fasta")
rbp_records=[]
for r in prot_records:
    if r.id in list(rb_annot['Gene_ID']):
        temp_name = list(rb_annot[rb_annot['Gene_ID']==r.id]['name'])[0]
        ID = '>rba|'+r.id+'|'+temp_name
        rbp_records.append( SeqRecord(Seq(str(r.seq)), id = ID, name="",description="") )

In [76]:
SeqIO.write( rbp_records, '../LP_HMX2/ribosomal_proteins.fasta' ,"fasta")

60